----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Delivery - EndPoint - Overview

This notebook provides an overview of using the Delivery Layer's Endpoint interface to retrieve LSEG Data Platform content directly from an Endpoint.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the LD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [1]:
import lseg.data as ld
from lseg.data.delivery import endpoint_request
import datetime
import json
import asyncio
import pandas as pd

## Open the default session

To open the default session ensure you have a '*lseg-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x2df116dfc10 {name='workspace'}>

## Defining an Endpoint
Defining an Endpoint
At the heart of the Endpoint Request interfaces, is the data endpoint URL.  For example, the following data services are represented by their endpoint URLs:

* Historical Pricing: https://api.refinitiv.com/data/historical-pricing/v1/views/events
* News Headlines: https://api.refinitiv.com/data/news/v1/headlines 
* Symbology Lookup: https://api.refinitiv.com/discovery/symbology/v1/lookup 
* Basic ESG scores: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic

When creating a request to retrieve data from an endpoint, the first step is to specify the endpoint URL, for example:

In [3]:
esg_url = 'https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic'

### Defining Endpoint Request Properties
Depending on the service and how you choose to prepare your endpoint URL, the interfaces support the ability to specify additional properties to be included within your request.  The following request features are available:

* Query Parameters Endpoint: URLs supporting query strings are represented as field=value pairs.
Eg: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic?universe=IBM.N

* Path Parameters : In some cases, URLs define parameters that are embedded within the URL.  When defining the URL as part of your application, you must enclose the path parameter(s) within curly braces.
Eg: https://api.refinitiv.com/data/historical-pricing/v1/views/events/{universe}

* Header Parameters : Endpoints that enforce different content types or formats, typically require the specification within the header portion of the HTTP request.  For example, the endpoint to retrieve a news story offers the ability to specify the response format as either "text/html" or "application/json", which is specified within the HTTP header of the request.

* Body Parameters : Many of the RDP endpoints support the HTTP POST defining the ability to specify a body of data in the request.  The libraries support the ability to specify JSON details within the support endpoint request.  

The following simple code segments demonstrate the above mechanisms.

**Query Parameter:**

In [4]:
# Example request with Query Parameter - ESG
request_definition = ld.delivery.endpoint_request.Definition(
    url = esg_url,
    method = ld.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "IBM.N"}
)
response = request_definition.get_data()
response.data.raw

{}

**Path Parameter:**

In [5]:
# Example request with Path parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
#storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
storyId = "urn:newsml:reuters.com:20210805:nZON003MH6:1"
request_definition = ld.delivery.endpoint_request.Definition(
    url = story_url,
    method = ld.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId}
)
response = request_definition.get_data()
response.data.raw

{'status': 200,
 'newsItem': {'_version': 0,
  'contentMeta': {'audience': [],
   'creator': [{'_qcode': 'NS:RTRS', '_role': 'sRole:source'}],
   'infoSource': [{'_qcode': 'NS:RTRS', '_role': 'sRole:source'},
    {'_qcode': 'NS:RTRS', '_role': 'sRole:origProv'}],
   'language': [{'_tag': 'fr'}],
   'subject': [{'_qcode': 'G:4'},
    {'_qcode': 'E:5'},
    {'_qcode': 'E:9'},
    {'_qcode': 'G:6J'},
    {'_qcode': 'G:9'},
    {'_qcode': 'E:C'},
    {'_qcode': 'M:1QD'},
    {'_qcode': 'M:MR'},
    {'_qcode': 'M:AJ'},
    {'_qcode': 'M:1A'},
    {'_qcode': 'M:1RT'},
    {'_qcode': 'M:N'},
    {'_qcode': 'M:2CY'},
    {'_qcode': 'I:1H'},
    {'_qcode': 'M:2CQ'},
    {'_qcode': 'M:Z'},
    {'_qcode': 'I:1P'},
    {'_qcode': 'M:E7'},
    {'_qcode': 'E:1'},
    {'_qcode': 'M:NP'},
    {'_qcode': 'L:fr'},
    {'_qcode': 'M:1ZT'},
    {'_qcode': 'M:2B'},
    {'_qcode': 'Meta:NewsRoom'},
    {'_qcode': 'P:67268310087'},
    {'_qcode': 'P:66744539837'},
    {'_qcode': 'P:66977754594'}],
   'urgenc

**Header Parameter:**

In [6]:
# Example request with Path + Header parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
request_definition = ld.delivery.endpoint_request.Definition(
    url = story_url,
    method = ld.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId},
    header_parameters = {"accept": "text/html"}
)
response = request_definition.get_data()
response.data.raw

{}

**Body Parameter:**

In [7]:
# Example request with Body Parameter - Symbology Lookup
lookup_url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup'
request_definition = ld.delivery.endpoint_request.Definition(
    url = lookup_url,
    method = ld.delivery.endpoint_request.RequestMethod.POST,
    body_parameters = {
        "from": [
            {
                "identifierTypes": [
                    "RIC"
                ],
                "values": [
                    "MSFT.O",
                    "IBM.N"
                ]
            }
        ],
        "to": [
            {
                "identifierTypes": [
                    "LEI",
                    "ISIN",
                    "ExchangeTicker"
                ]
            }
        ],
        "type": "auto"
    }
)
response = request_definition.get_data()
response.data.raw

{'data': [{'input': [{'value': 'MSFT.O', 'identifierType': 'RIC'}],
   'output': [{'value': 'INR2EJN1ERAN0W5ZP974', 'identifierType': 'LEI'},
    {'value': 'MSFT', 'identifierType': 'ExchangeTicker'}],
   'errors': ['User is not entitled to receive the ISIN identifier back'],
   'errorCodes': ['USER_NOT_ENTITLED_FOR_IDENTIFIER']},
  {'input': [{'value': 'IBM.N', 'identifierType': 'RIC'}],
   'output': [{'value': 'IBM', 'identifierType': 'ExchangeTicker'},
    {'value': 'VGRQXHF3J8VDLUA7XE92', 'identifierType': 'LEI'}],
   'errors': ['User is not entitled to receive the ISIN identifier back'],
   'errorCodes': ['USER_NOT_ENTITLED_FOR_IDENTIFIER']}],
 'requestId': 'f8b204ff-738e-419b-a9d8-5dae819917e3',
 'effectiveAt': '2024-06-19T16:51:31.784Z',
 'messages': ['This account is not entitled to all requested identifier types'],
 'errorCodes': ['ACCOUNT_NOT_ENTITLED_TO_IDENTIFIER_TYPES']}

### How to send several request with asyncio.gather

The above examples represent simple requests to retrieve the data synchronously from the platform.  While this workflow may be suitable for simple applications or when demonstrating functionality, multi-threaded applications, including those that are GUI/Windows-based, will require asynchronous behavior. For example, asyncio.gather allows us to send several request simultaneously (multi-threaded requests).

#### Example: request ESG data for two instruments

In [8]:
request_definition_1 = ld.delivery.endpoint_request.Definition(
    url = esg_url,
    method = ld.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "IBM.N"}
)
request_definition_2 = ld.delivery.endpoint_request.Definition(
    url = esg_url,
    method = ld.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "BT.L"}
)

response = request_definition_1.get_data_async()
tasks = asyncio.gather(
    request_definition_1.get_data_async(),
    request_definition_2.get_data_async()
)

# Wait for all tasks to be completed
await tasks
ibm,bt = tasks.result()

### Checking the Response Data
When using ***async*** calls, we should check the response's ***is_success*** flag to ensure it worked. If it failed, the **errors** should provide a reason for the failure.

In [10]:
def display_reponse(response):
    print(response)
    print("\nReponse received for", response.closure)
    if response.is_success:
        display(response.data.raw)
    else:
        print(response.errors)

display_reponse(ibm)
display_reponse(bt)

Response(is_success=False, raw=<Response [403 Forbidden]>, errors=[Error(code=403, message='access denied. Scopes required to access the resource: [trapi.data.esg.views-basic.read] or [trapi.demo-access]. Missing scopes: [trapi.data.esg.views-basic.read trapi.demo-access]. Contact LSEG to check your permissions.')], closure=None, requests_count=1, _data_factory=<lseg.data.delivery._data._response_factory.ResponseFactory object at 0x000002DF117A45D0>, _kwargs={'session': <lseg.data.session.Definition object at 0x2df116dfc10 {name='workspace'}>, 'method': <RequestMethod.GET: 'GET'>, 'path_parameters': None, 'query_parameters': {'universe': 'IBM.N'}, 'header_parameters': None, 'body_parameters': None}, _data_raw={})

Reponse received for None
[Error(code=403, message='access denied. Scopes required to access the resource: [trapi.data.esg.views-basic.read] or [trapi.demo-access]. Missing scopes: [trapi.data.esg.views-basic.read trapi.demo-access]. Contact LSEG to check your permissions.')]

In [11]:
ld.close_session()